In [1]:
import dlib
import cv2
import os
import re
import json
from pylab import *
from PIL import Image, ImageChops, ImageEnhance
import tqdm

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')

if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [4]:
train_frame_folder = 'train_sample_videos'
list_of_train_data = [f for f in os.listdir(train_frame_folder) if f.endswith('.mp4')]

with open(os.path.join(train_frame_folder, 'metadata.json'), 'r') as file:
    data = json.load(file)

In [5]:
data[list_of_train_data[0]]['label']

print(len(list_of_train_data))
print(list_of_train_data[0:10])

'FAKE'

400
['aagfhgtpmv.mp4', 'aapnvogymq.mp4', 'abarnvbtwb.mp4', 'abofeumbvv.mp4', 'abqwwspghj.mp4', 'acifjvzvpm.mp4', 'acqfdwsrhi.mp4', 'acxnxvbsxk.mp4', 'acxwigylke.mp4', 'aczrgyricp.mp4']


In [6]:
# DLib Histogram of Oriented Gradients (HOG)
detector = dlib.get_frontal_face_detector()

# DLib Convolutional Neural Network (CNN)
# https://towardsdatascience.com/cnn-based-face-detector-from-dlib-c3696195e01c
# detector = dlib.cnn_face_detection_model_v1(model.path)

In [8]:
# Check test
count = 0
cap = cv2.VideoCapture('train_sample_videos/aagfhgtpmv.mp4')
frameRate = cap.get(5) # fps
print(frameRate)
while cap.isOpened():
    frameId = cap.get(1) # Index of frame
    #print(frameId)
    ret, frame = cap.read()
    if ret != True:
        break

    # Detect faces per 30 frames
    if frameId % ((int(frameRate)+1)*1) == 0:
        face_rects, scores, idx = detector.run(frame, 0)
        print(face_rects, scores, idx)

29.97
rectangles[[(337, 194) (595, 452)]] [1.2877205406329062] [2]
rectangles[[(509, 194) (767, 452)]] [2.3029888664385703] [2]
rectangles[[(624, 222) (882, 480)]] [3.0051868595613356] [0]
rectangles[[(973, 281) (1188, 496)]] [2.23999053616656] [0]
rectangles[[(1082, 194) (1340, 452)]] [2.3104915772743646] [1]
rectangles[[(997, 233) (1212, 448)]] [2.2213115845985834] [1]
rectangles[[(1021, 161) (1236, 376)]] [1.6856887974275465] [0]
rectangles[[(902, 161) (1117, 376)]] [1.5005801357759352] [0]
rectangles[[(1021, 185) (1236, 400)]] [2.8618996777070875] [0]
rectangles[[(687, 209) (902, 424)]] [2.1783922352327223] [0]


In [8]:
#for vid in list_of_train_data:
for vid in tqdm.tqdm(list_of_train_data):
    count = 0
    cap = cv2.VideoCapture(os.path.join(train_frame_folder, vid))
    frameRate = cap.get(5) # 29.97
    while cap.isOpened():
        frameId = cap.get(1)
        ret, frame = cap.read()
        if ret != True:
            break

        # per 30 frames
        if frameId % ((int(frameRate)+1)*1) == 0:
            face_rects, scores, idx = detector.run(frame, 0)
            #print(frameId, len(face_rects))
            for i, d in enumerate(face_rects):
                x1 = d.left()
                y1 = d.top()
                x2 = d.right()
                y2 = d.bottom()
                if(x1<0): x1=0
                if(x2<0): x2=0
                if(y1<0): y1=0
                if(y2<0): y2=0
                crop_img = frame[y1:y2, x1:x2]
                #print(count, x1, y1, x2, y2)
                if data[vid]['label'] == 'REAL':
                    cv2.imwrite('dataset/real/'+vid.split('.')[0]+'_'+str(count)+'.png', cv2.resize(crop_img, (128, 128)))
                elif data[vid]['label'] == 'FAKE':
                    cv2.imwrite('dataset/fake/'+vid.split('.')[0]+'_'+str(count)+'.png', cv2.resize(crop_img, (128, 128)))
                count+=1

  0%|                                                                                          | 0/400 [00:00<?, ?it/s]

True

True

True

True

...

True

True

True

True

100%|████████████████████████████████████████████████████████████████████████████████| 400/400 [23:14<00:00,  3.49s/it]
